# Create CSV file for Review data


In [1]:
import json
from amazon_scraper import AmazonScraper

az = AmazonScraper()
rtn = az.scrape_asins_concurrently(
    ["0596514522", "0609806483", "0071455531", "0764120964", "1853672831"]
)
print(json.dumps(rtn, indent=4))

Scraping reviews sorted by: recentScraping reviews sorted by: recent
Scraping page 1 for sort by recent and star rating all_star and format type all_formats
URL: https://www.amazon.com/product-reviews/0609806483?sortBy=recent&pageNumber=1&filterByStar=all_star&formatType=all_formats&mediaType=media_reviews_only

Scraping page 1 for sort by recent and star rating all_star and format type all_formats
URL: https://www.amazon.com/product-reviews/0596514522?sortBy=recent&pageNumber=1&filterByStar=all_star&formatType=all_formats&mediaType=media_reviews_only
Scraping reviews sorted by: recent
Scraping page 1 for sort by recent and star rating all_star and format type all_formats
URL: https://www.amazon.com/product-reviews/0071455531?sortBy=recent&pageNumber=1&filterByStar=all_star&formatType=all_formats&mediaType=media_reviews_only
Scraping reviews sorted by: recent
Scraping page 1 for sort by recent and star rating all_star and format type all_formats
URL: https://www.amazon.com/product-revi

In [2]:
# Write the results to a JSON file
with open("sample_2.json", "w") as json_file:
  json.dump(rtn, json_file, indent=4)

print("Scraped data has been written to sample_2.json")

Scraped data has been written to sample_2.json
